In [40]:
import pandas as pd
from deviation_functions import normalise, calc_dev_scores, score_ds_q123

In [41]:
ri_data_path = "../data/reference_data/"
mdd_data_path = "../data/depression_data/"

In [42]:
pd.read_csv(ri_data_path+"reference_descriptives.csv", index_col=0)

,dataset,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
2,clac_males,"(3.686937999154304, 0.49415676542521314)","(4.3263330412651735, 0.4461043928532965)","(0.8518480106705816, 0.06863502555077977)","(0.24725776991129317, 0.0940298671910015)","(0.5927437801552339, 0.20629731695112002)","(117.81970727376503, 29.354948559032835)","(3.117727471894877, 1.38293729213957)","(71.83124907753657, 4.869715529612232)","(1.6094610277037598, 126.38332346541796)","(5.357098162890241, 2.147612916150901)","(-16.188623839509784, 4.83573795115372)","(-0.006795774336491557, 0.0018013356878964013)","(8.648310687116682, 1.705465684045437)","(481.08365873890375, 71.61009609075668)","(201.03667683428932, 65.16900811762754)","(168.7434337011567, 68.98843235778135)","(200.99299430959064, 68.71150803959135)","(1510.1585360480178, 104.34301160030348)","(426.8070876071593, 61.442693673022255)","(313.1959398886768, 92.84191248771229)","(409.58946847774286, 109.1338594022463)","(531.459005482349, 233.80467598334266)","(398.2230345999431, 147.61321435854677)","(4.6553844239925715, 1.613400771901009)","(73.65625248231085, 59.041679437498175)"
3,clac_females,"(3.8276591254940353, 0.5265453411037165)","(4.394874192454035, 0.43751427635886153)","(0.869766928973627, 0.07332294578815503)","(0.2263646563662828, 0.10638196912567187)","(0.5508401458511046, 0.16398355621075136)","(195.7247923658587, 27.943870668467966)","(3.4923637662152753, 0.9728680275805698)","(73.44376443332233, 4.334449618285865)","(6.693620524779084, 123.31132404447959)","(9.439438442630854, 2.376319381688551)","(-15.87947760219253, 4.6561875899956116)","(-0.006158778927092862, 0.0017534436661733976)","(9.43607437610024, 1.5597188027063704)","(529.0011814430956, 63.95079988949581)","(203.8570423045786, 48.869218033388925)","(192.5716438740452, 53.44909453296275)","(268.00814987790824, 65.78456521172112)","(1607.980525188241, 141.30641568052218)","(501.589605647992, 85.41419432721061)","(446.5019042102586, 107.14425920907519)","(602.0584694531924, 130.21775516593894)","(566.1186377158197, 200.86348848189147)","(395.542366651839, 135.06204333339437)","(5.120149074391844, 1.6480290318929234)","(99.63288161368564, 75.31693788140147)"
4,common_males,"(3.969768703489275, 0.901547763188605)","(4.14650563441203, 0.8450807405830122)","(0.9568429018275012, 0.09173133500449708)","(0.08469267301183372, 0.1655812330911897)","(0.13132122274826113, 0.3455700334194028)","(128.51757458221658, 32.05163820145666)","(3.153468031224966, 1.4284810493021933)","(66.69940961690456, 8.178476484494288)","(-3.3333156733543454, 506.3943141788426)","(5.952800804167977, 2.53707792079255)","(-17.42129886583369, 4.974427981655563)","(-0.00642580361997799, 0.0018862228854504288)","(8.15561221692547, 1.5911861525931466)","(472.32366313176055, 79.62660301894766)","(193.5970062978888, 87.75800918174676)","(194.37190973253286, 82.85549366617997)","(205.79217118293846, 89.27391886604313)","(1535.6135731288005, 160.31314446562854)","(391.0939583411243, 88.40055787766869)","(363.921357774823, 121.96755333980647)","(416.1733777710544, 153.25615811493137)","(461.0397125382718, 174.63055576708902)","(382.61983359243504, 151.15198827316894)","(5.341484963548701, 1.8855932204272083)","(86.37788670064607, 71.66176034186762)"
5,common_females,"(4.056499926287583, 0.8767711418891906)","(4.242115495493237, 0.8264258916605276)","(0.9564765209233418, 0.08845690867537337)","(0.08634382096004158, 0.16189914620227272)","(0.13297826564215148, 0.2507070053873603)","(204.3200383725392, 36.61263439862343)","(3.3360476657417286, 1.2471963005052253)","(65.85694812509153, 9.130588027860236)","(1.8508750115616577, 110.27091121232054)","(9.655722725941352, 2.

### Load data

In [43]:
# ri datasets
clac_female_reference_intervals = pd.read_csv(ri_data_path + "clac_female_reference_intervals.csv", index_col=0)
clac_male_reference_intervals = pd.read_csv(ri_data_path + "clac_male_reference_intervals.csv",index_col=0)

common_female_reference_intervals = pd.read_csv(ri_data_path + "common_female_reference_intervals.csv",index_col=0)
common_male_reference_intervals = pd.read_csv(ri_data_path + "common_male_reference_intervals.csv",index_col=0)

In [44]:
## checking the reference interval dfs
clac_male_reference_intervals.head()

,feature,RI_lower_limit,RI_upper_limit,CI_for_lower_limit,CI_for_upper_limit,ci_lower_width/ri_width,ci_upper_width/ri_width,valid_CI_lower_limit,valid_CI_upper_limit,median,q1,q3
0,Speaking_Rate,-1.883544,1.862419,"(-2.103771545725876, -1.6563509045876526)","(1.715459199614647, 2.2428457041148167)",0.119441,0.140788,True,True,-0.025285,-0.645892,0.722881
1,Articulation_Rate,-1.820120,2.151624,"(-2.0790442854931874, -1.5476706161395857)","(1.8807474517867075, 2.3347471494671304)",0.133789,0.114307,True,True,0.006440,-0.650212,0.620862
2,Phonation_Ratio,-2.410274,1.592982,"(-2.518333166338251, -2.08599944367207)","(1.4050946767415433, 2.160929076097077)",0.107996,0.188805,True,True,0.071917,-0.494887,0.681912
3,Pause_Rate,-1.754677,2.143551,"(-2.632467090556839, -1.6750344803969097)","(1.8802912395459805, 2.4921166510584367)",0.245607,0.156950,False,True,-0.038674,-0.674937,0.529467
4,Mean_Pause_Duration,-1.116823,1.992023,"(-2.8764196783796816, -1.0647374936862448)","(1.7045481314880082, 2.7915574423040717)",0.582751,0.349650,False,False,-0.090958,-0.528997,0.384608


Load in RADAR data and split into male and female for normalisation

In [45]:
mdd = pd.read_csv(mdd_data_path + "synthetic_mdd_data.csv", index_col=0)

# 0 = males, 1 = females
mdd_males = mdd.loc[mdd["gender"] == 0,:]
mdd_females = mdd.loc[mdd["gender"] == 1,:]

RADAR descriptives

In [46]:
features_for_descriptives = ["speaking_rate", "mean_f0", "cepstral_peak_prominence", "mean_f1_loc", "mean_f2_loc", "spectral_gravity"]
mdd_males[features_for_descriptives].describe().map(lambda x: f"{x:0.2f}")

,speaking_rate,mean_f0,cepstral_peak_prominence,mean_f1_loc,mean_f2_loc,spectral_gravity
count,494.00,494.00,494.00,494.00,494.00,494.00
mean,4.08,188.42,0.82,598.90,1657.15,2738.54
std,1.16,62.72,0.36,175.74,482.69,1275.89
min,2.02,80.58,0.20,302.71,802.66,503.78
25%,3.03,135.61,0.53,444.70,1268.65,1651.52
50%,4.20,184.09,0.79,608.95,1632.82,2764.90
75%,5.04,244.43,1.12,756.57,2050.41,3788.50
max,5.98,299.80,1.49,899.24,2497.94,4996.15


In [47]:
mdd_females[features_for_descriptives].describe().map(lambda x: f"{x:0.2f}")

,speaking_rate,mean_f0,cepstral_peak_prominence,mean_f1_loc,mean_f2_loc,spectral_gravity
count,506.00,506.00,506.00,506.00,506.00,506.00
mean,4.01,188.69,0.89,603.12,1645.97,2758.30
std,1.17,65.33,0.36,170.30,507.06,1310.94
min,2.01,80.21,0.21,302.50,804.87,507.24
25%,3.02,128.31,0.60,462.74,1216.25,1581.22
50%,4.01,188.02,0.93,605.72,1630.90,2695.26
75%,5.05,247.95,1.21,748.22,2113.57,3883.72
max,6.00,299.81,1.50,899.13,2496.10,4990.50


### Normalising RADAR Data

mean_sds contains tuples of the mean and sd for each speech feature for each of the three reference interval datasets, separated by male and female. We need this to normalise the RADAR data using reference data.

In [48]:
ref_descriptives = pd.read_csv(ri_data_path + "reference_descriptives.csv", index_col=0)

In [49]:
ref_descriptives

,dataset,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
2,clac_males,"(3.686937999154304, 0.49415676542521314)","(4.3263330412651735, 0.4461043928532965)","(0.8518480106705816, 0.06863502555077977)","(0.24725776991129317, 0.0940298671910015)","(0.5927437801552339, 0.20629731695112002)","(117.81970727376503, 29.354948559032835)","(3.117727471894877, 1.38293729213957)","(71.83124907753657, 4.869715529612232)","(1.6094610277037598, 126.38332346541796)","(5.357098162890241, 2.147612916150901)","(-16.188623839509784, 4.83573795115372)","(-0.006795774336491557, 0.0018013356878964013)","(8.648310687116682, 1.705465684045437)","(481.08365873890375, 71.61009609075668)","(201.03667683428932, 65.16900811762754)","(168.7434337011567, 68.98843235778135)","(200.99299430959064, 68.71150803959135)","(1510.1585360480178, 104.34301160030348)","(426.8070876071593, 61.442693673022255)","(313.1959398886768, 92.84191248771229)","(409.58946847774286, 109.1338594022463)","(531.459005482349, 233.80467598334266)","(398.2230345999431, 147.61321435854677)","(4.6553844239925715, 1.613400771901009)","(73.65625248231085, 59.041679437498175)"
3,clac_females,"(3.8276591254940353, 0.5265453411037165)","(4.394874192454035, 0.43751427635886153)","(0.869766928973627, 0.07332294578815503)","(0.2263646563662828, 0.10638196912567187)","(0.5508401458511046, 0.16398355621075136)","(195.7247923658587, 27.943870668467966)","(3.4923637662152753, 0.9728680275805698)","(73.44376443332233, 4.334449618285865)","(6.693620524779084, 123.31132404447959)","(9.439438442630854, 2.376319381688551)","(-15.87947760219253, 4.6561875899956116)","(-0.006158778927092862, 0.0017534436661733976)","(9.43607437610024, 1.5597188027063704)","(529.0011814430956, 63.95079988949581)","(203.8570423045786, 48.869218033388925)","(192.5716438740452, 53.44909453296275)","(268.00814987790824, 65.78456521172112)","(1607.980525188241, 141.30641568052218)","(501.589605647992, 85.41419432721061)","(446.5019042102586, 107.14425920907519)","(602.0584694531924, 130.21775516593894)","(566.1186377158197, 200.86348848189147)","(395.542366651839, 135.06204333339437)","(5.120149074391844, 1.6480290318929234)","(99.63288161368564, 75.31693788140147)"
4,common_males,"(3.969768703489275, 0.901547763188605)","(4.14650563441203, 0.8450807405830122)","(0.9568429018275012, 0.09173133500449708)","(0.08469267301183372, 0.1655812330911897)","(0.13132122274826113, 0.3455700334194028)","(128.51757458221658, 32.05163820145666)","(3.153468031224966, 1.4284810493021933)","(66.69940961690456, 8.178476484494288)","(-3.3333156733543454, 506.3943141788426)","(5.952800804167977, 2.53707792079255)","(-17.42129886583369, 4.974427981655563)","(-0.00642580361997799, 0.0018862228854504288)","(8.15561221692547, 1.5911861525931466)","(472.32366313176055, 79.62660301894766)","(193.5970062978888, 87.75800918174676)","(194.37190973253286, 82.85549366617997)","(205.79217118293846, 89.27391886604313)","(1535.6135731288005, 160.31314446562854)","(391.0939583411243, 88.40055787766869)","(363.921357774823, 121.96755333980647)","(416.1733777710544, 153.25615811493137)","(461.0397125382718, 174.63055576708902)","(382.61983359243504, 151.15198827316894)","(5.341484963548701, 1.8855932204272083)","(86.37788670064607, 71.66176034186762)"
5,common_females,"(4.056499926287583, 0.8767711418891906)","(4.242115495493237, 0.8264258916605276)","(0.9564765209233418, 0.08845690867537337)","(0.08634382096004158, 0.16189914620227272)","(0.13297826564215148, 0.2507070053873603)","(204.3200383725392, 36.61263439862343)","(3.3360476657417286, 1.2471963005052253)","(65.85694812509153, 9.130588027860236)","(1.8508750115616577, 110.27091121232054)","(9.655722725941352, 2.

The ref_descriptives dataset was exported with the columns as strings, so they need to be converted back to tuples.

In [51]:
# convert strings to tuples
import ast

for col in ref_descriptives.columns[1:]:
    ref_descriptives[col] = ref_descriptives[col].apply(ast.literal_eval)

# columns to lowercase for compatibility with radar
ref_descriptives.columns = map(str.lower, ref_descriptives.columns)

In [13]:
# should all be tuples
for col in mean_sds.columns[1:]:
    print(col)
    for i in mean_sds[col]:
        print(type(i))

speaking_rate
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
articulation_rate
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
phonation_ratio
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
pause_rate
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
mean_pause_duration
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
mean_f0
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
stdev_f0_semitone
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
mean_db
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
range_ratio_db
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
hnr_db
<

The normalise function will use the mean and sd from each feature column of the reference dataset to normalise the radar data. 

In [14]:
# function to normalise
def normalise(df, features_to_normalise, mean_sd_df, reference_dataset):
    """
    df: pandas dataframe which contains features to normalise using reference data.
    features_to_normalise: an array of the feature columns to normalise. 
    mean_sd_df: a df with a row per reference dataset and a column for each of the features_to_normalise. The features should have a tuple of the (mean, sd) of that feature for the given dataset.
    reference_data_set: the dataset (row) in mean_sd_df to use for normalisation. 
    """
    if "dataset" not in mean_sd_df.columns:
        assert "the mean_sd_df column with the dataset names should be named 'dataset"
    
    normalised_df = df.copy()

    for feature in features_to_normalise:
        # find the mean and sd from the correct dataset for the correct feature
        feature_values = mean_sd_df.loc[mean_sd_df["dataset"] == reference_dataset, feature].iloc[0]
        mean, sd = feature_values[0], feature_values[1]
        # standardise
        normalised_df.loc[:, feature] = (normalised_df[feature] - mean)/sd


    print(f"[INFO] Successfully normalised with {reference_dataset}")
    return normalised_df

In [15]:
speech_features = [
    'speaking_rate', 'articulation_rate', 'phonation_ratio', 'pause_rate', 'mean_pause_duration', 'mean_f0', 'stdev_f0_semitone', 'mean_db', 'range_ratio_db', 'hnr_db', 'spectral_slope', 
    'spectral_tilt', 'cepstral_peak_prominence', 'mean_f1_loc', 'std_f1_loc', 'mean_b1_loc', 'std_b1_loc', 'mean_f2_loc', 'std_f2_loc', 'mean_b2_loc', 'std_b2_loc', 'spectral_gravity', 
    'spectral_std_dev', 'spectral_skewness', 'spectral_kurtosis'
    ]

It is a wise idea to check that the raw mean/sd are somewhat similar before normalisation. 

In [16]:
radar_datasets = [(radar_males, "radar_males"), (radar_females, "radar_females")]
descriptives_comparison = mean_sds.copy()

# want to add a row with the (mean, sd) of each feature for each of the 6 dfs.
for df in radar_datasets:
    data = df[0]
    data_name = df[1]
    
    data_row = {"dataset": data_name}
    
    for feature in speech_features:
        data_mean = data[feature].mean()
        data_std = data[feature].std()
        
        data_row[feature] = (data_mean, data_std)
    
    # print(data_row)
    descriptives_comparison = pd.concat([descriptives_comparison, pd.DataFrame([data_row])])

descriptives_comparison

,dataset,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
0,mdc_males,"(3.1918662800625794, 0.6532963349497485)","(4.356648844557102, 0.5135412508147292)","(0.7307296571290134, 0.11763399955621372)","(0.31288289080944, 0.0931458887151525)","(0.8913033624671574, 0.5151869261492393)","(117.18263156265584, 28.807046980878326)","(3.0028383290450975, 1.2823418279606889)","(68.26757149561018, 4.446323240570879)","(3.745765236856924, 58.2058347991454)","(6.493080542018815, 2.282408872999886)","(-18.342635957277732, 4.298507327413223)","(-0.006538770583198368, 0.0017167375379883948)","(8.243838934345925, 1.5717088282566105)","(452.80095658327957, 68.7085445594476)","(179.82213537399662, 59.7511832988767)","(162.66595769642043, 67.34258763925276)","(202.16439648098722, 59.602895269373256)","(1437.1604899542197, 112.4321879854814)","(419.1801895227432, 69.1822706312527)","(348.3858254222183, 97.24786620547415)","(433.60552144408217, 87.64308520915421)","(438.6701928850464, 151.44254381834222)","(321.83166834920013, 110.96652042902852)","(5.41458800810741, 1.5157678770131864)","(94.17813418932647, 61.13167550141343)"
1,mdc_females,"(3.4610264666789834, 0.6525607467215228)","(4.503919886149819, 0.4830645892830357)","(0.7668539061382968, 0.11204867786275624)","(0.2858364665799249, 0.08602490727713605)","(0.8337824578551096, 0.4459435542015938)","(186.91451154239476, 29.31407739327904)","(3.405480304496443, 1.1409742321220064)","(68.05872146577508, 4.735265122133672)","(-0.13389343467549447, 198.25282136309116)","(10.143700206349536, 2.608631071131703)","(-17.950099336690627, 4.298362432741766)","(-0.006356339926305415, 0.0015966578415241195)","(9.138983381748867, 1.5917675684379557)","(496.3212977637988, 56.089384959180386)","(186.44542815755915, 48.67628361766591)","(180.8947287030289, 51.07076386694902)","(245.38435986684698, 52.97486340034953)","(1546.884952657287, 127.8716525754859)","(477.29021073613563, 76.31898922075439)","(410.92594762726634, 95.0065925491667)","(549.9780296493958, 107.31881446487264)","(478.3074651117933, 122.74836579426436)","(330.29979798574533, 110.7604847651829)","(5.462908255585334, 1.5146103562754722)","(106.75792669254844, 71.3752473537651)"
2,clac_males,"(3.686937999154304, 0.49415676542521314)","(4.3263330412651735, 0.4461043928532965)","(0.8518480106705816, 0.06863502555077977)","(0.24725776991129317, 0.0940298671910015)","(0.5927437801552339, 0.20629731695112002)","(117.81970727376503, 29.354948559032835)","(3.117727471894877, 1.38293729213957)","(71.83124907753657, 4.869715529612232)","(1.6094610277037598, 126.38332346541796)","(5.357098162890241, 2.147612916150901)","(-16.188623839509784, 4.83573795115372)","(-0.006795774336491557, 0.0018013356878964013)","(8.648310687116682, 1.705465684045437)","(481.08365873890375, 71.61009609075668)","(201.03667683428932, 65.16900811762754)","(168.7434337011567, 68.98843235778135)","(200.99299430959064, 68.71150803959135)","(1510.1585360480178, 104.34301160030348)","(426.8070876071593, 61.442693673022255)","(313.1959398886768, 92.84191248771229)","(409.58946847774286, 109.1338594022463)","(531.459005482349, 233.80467598334266)","(398.2230345999431, 147.61321435854677)","(4.6553844239925715, 1.613400771901009)","(73.65625248231085, 59.041679437498175)"
3,clac_females,"(3.8276591254940353, 0.5265453411037165)","(4.394874192454035, 0.43751427635886153)","(0.869766928973627, 0.07332294578815503)","(0.2263646563662828, 0.10638196912567187)","(0.5508401458511046, 0.16398355621075136)","(195.7247923658587, 27.943870668467966)","(3.4923637662152753, 0.9728680275805698)","(73.44376443332233, 4.334449618285865)","(6.693620524779084, 123.31132404447959)","(9.439438442630854, 2.37

Now to create the normalised radar datasets. 

In [17]:
mdc_males_radar = normalise(radar_males, speech_features, mean_sds, "mdc_males")
mdc_females_radar = normalise(radar_females, speech_features, mean_sds, "mdc_females")

clac_males_radar = normalise(radar_males, speech_features, mean_sds, "clac_males")
clac_females_radar = normalise(radar_females, speech_features, mean_sds, "clac_females")

common_males_radar = normalise(radar_males, speech_features, mean_sds, "common_males")
common_females_radar = normalise(radar_females, speech_features, mean_sds, "common_females")

[INFO] Successfully normalised with mdc_males
[INFO] Successfully normalised with mdc_females
[INFO] Successfully normalised with clac_males
[INFO] Successfully normalised with clac_females
[INFO] Successfully normalised with common_males
[INFO] Successfully normalised with common_females
[INFO] Successfully normalised with mdc_males
[INFO] Successfully normalised with mdc_females
[INFO] Successfully normalised with clac_males
[INFO] Successfully normalised with clac_females
[INFO] Successfully normalised with common_males
[INFO] Successfully normalised with common_females


### Check that the normalised radar datasets look as they should

In [18]:
mdc_males_radar.head(3)

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,0.964019,0.107325,1.151983,-1.649533,-0.099582,0.162931,0.468736,0.067304,-0.068850,-0.675942,-1.011972,-0.231692,-1.025686,-0.869650,-0.372774,-0.109521,-0.568982,-0.774815,-0.453370,0.323585,-0.054208,-0.323620,-0.428314,1.137664,0.716916
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,0.450316,-0.034673,0.618216,-0.685794,-0.197928,0.567846,0.263330,0.788721,-0.069139,-0.256101,-0.651579,-1.973517,-0.177969,0.536488,-0.187816,1.499333,0.459310,-0.935952,-1.696820,0.281951,0.696637,0.185011,0.013453,-1.086207,-0.874647
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,0.934362,-0.886937,2.073565,-2.508631,-1.108924,0.741481,0.603414,0.724228,-0.069035,0.064203,0.262732,0.884800,-0.508174,0.285565,-0.617847,1.576813,0.571247,-0.464754,-1.250007,-0.143735,-0.450011,0.346897,0.653200,0.265500,-0.486107


In [19]:
mdc_females_radar.head(3)

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-08-28,RADAR-MDD-KCL-s1,15,61,1,17,170,0.264641,-0.431386,0.705732,-0.154692,-0.601980,-1.467744,-0.501668,-0.341169,-0.000609,-0.187077,-0.519217,0.755915,-1.105373,-0.472461,1.837605,3.301237,0.475759,0.060903,-1.892434,-0.021261,-1.231005,-1.574688,-0.107797,1.976993,0.754335
2,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-10-23,RADAR-MDD-KCL-s1,12,61,1,17,170,0.730624,-0.182499,1.114757,-0.975274,-0.667758,-2.070856,-0.811641,-0.134975,-0.000619,-0.702044,-1.641160,1.167000,-1.685141,0.229168,1.420935,2.817064,0.379039,-0.069879,-0.943200,1.549523,-1.718221,-1.053658,0.368712,1.374837,0.532727
3,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-12-04,RADAR-MDD-KCL-s1,6,61,1,17,170,0.594955,-0.559777,1.270736,-0.967665,-0.865093,-1.524321,-0.097299,0.188893,-0.000671,-0.541970,-1.425060,-1.115197,-1.091969,1.613967,1.877523,3.061621,1.373935,-0.519214,-1.235251,0.009429,-1.509614,-1.283301,-0.408540,0.795811,0.457896


In [20]:
clac_males_radar.head(3)

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,0.272622,0.191506,0.209718,-0.936108,1.198543,0.138187,0.351564,-0.670352,-0.014806,-0.189417,-1.344982,-0.078137,-1.182405,-1.229367,-0.667315,-0.195003,-0.476507,-1.534479,-0.634608,0.717971,0.176527,-0.606484,-0.839490,1.539381,1.089877
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,-0.406515,0.028043,-0.705109,0.018571,0.952943,0.535545,0.161099,-0.011658,-0.014938,0.256776,-1.024627,-1.738159,-0.401174,0.119796,-0.497733,1.375469,0.415470,-1.708108,-2.034689,0.674361,0.779515,-0.277027,-0.507397,-0.549916,-0.558026
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,0.233415,-0.953056,1.789224,-1.787130,-1.322091,0.705939,0.476445,-0.070543,-0.014891,0.597183,-0.211893,0.985920,-0.705481,-0.120960,-0.892014,1.451101,0.512569,-1.200381,-1.531594,0.228474,-0.141334,-0.172169,-0.026475,0.719995,-0.155732


In [21]:
clac_females_radar.head(3)

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-08-28,RADAR-MDD-KCL-s1,15,61,1,17,170,-0.368322,-0.227059,-0.325092,0.433950,0.088382,-1.854999,-0.677660,-1.615100,-0.056348,0.091001,-0.924019,0.575654,-1.318563,-0.925398,1.474060,2.935874,0.039211,-0.377250,-1.975410,-0.350890,-1.414480,-1.399466,-0.571457,2.024924,0.809458
2,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-10-23,RADAR-MDD-KCL-s1,12,61,1,17,170,0.209182,0.047739,0.299960,-0.229607,-0.090498,-2.487684,-1.041195,-1.389839,-0.056363,-0.474310,-1.959741,0.949981,-1.910244,-0.310020,1.059035,2.473245,-0.038676,-0.495598,-1.127254,1.041950,-1.816019,-1.081062,-0.180686,1.471517,0.599448
3,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-12-04,RADAR-MDD-KCL-s1,6,61,1,17,170,0.041044,-0.368817,0.538322,-0.223454,-0.627137,-1.914350,-0.203419,-1.036022,-0.056446,-0.298587,-1.760249,-1.128151,-1.304883,0.904547,1.513820,2.706920,0.762493,-0.902212,-1.388206,-0.323677,-1.644095,-1.221398,-0.818088,0.939366,0.528533


In [22]:
common_males_radar.head(3)

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,-0.164287,0.313886,-0.987676,0.450190,2.050753,-0.207209,0.315335,0.228333,0.006066,-0.395138,-1.059681,-0.270764,-0.957684,-0.995585,-0.410773,-0.471681,-0.420512,-1.157530,-0.037092,0.130628,0.082745,-0.408745,-0.716608,0.953301,0.720419
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,-0.536536,0.227596,-1.672166,0.992331,1.904135,0.156717,0.130943,0.620540,0.006032,-0.017441,-0.748257,-1.856079,-0.120344,0.217749,-0.284841,0.835949,0.266018,-1.270539,-1.010216,0.097432,0.512133,0.032348,-0.392289,-0.834398,-0.637277
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,-0.185778,-0.290310,0.194138,-0.033086,0.545993,0.312775,0.436235,0.585477,0.006044,0.270712,0.041817,0.745407,-0.446507,0.001231,-0.577634,0.898923,0.340751,-0.940075,-0.660540,-0.241978,-0.143604,0.172738,0.077374,0.252196,-0.305830


In [23]:
common_females_radar.head(3)

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-08-28,RADAR-MDD-KCL-s1,15,61,1,17,170,-0.482200,0.064636,-1.249719,1.150008,1.724543,-1.650553,-0.403270,0.064207,-0.019095,-0.000014,-0.478749,0.519982,-0.875036,-0.672502,1.255838,1.901572,0.078090,-0.226305,-1.040501,-0.479485,-0.886071,-1.302845,-0.415196,1.171008,0.407410
2,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-10-23,RADAR-MDD-KCL-s1,12,61,1,17,170,-0.135380,0.210116,-0.731606,0.713993,1.607541,-2.133437,-0.686844,0.171142,-0.019112,-0.489119,-1.488790,0.835839,-1.476506,-0.141961,0.946329,1.551219,0.024196,-0.313423,-0.345884,0.513725,-1.193967,-0.940549,-0.041086,0.782014,0.267220
3,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-12-04,RADAR-MDD-KCL-s1,6,61,1,17,170,-0.236355,-0.010411,-0.534026,0.718036,1.256533,-1.695851,-0.033342,0.339105,-0.019205,-0.337084,-1.294245,-0.917690,-0.861130,0.905163,1.285490,1.728183,0.578573,-0.612739,-0.559597,-0.460080,-1.062137,-1.100230,-0.651312,0.407962,0.219882


In [24]:
speech_features

['speaking_rate',
 'articulation_rate',
 'phonation_ratio',
 'pause_rate',
 'mean_pause_duration',
 'mean_f0',
 'stdev_f0_semitone',
 'mean_db',
 'range_ratio_db',
 'hnr_db',
 'spectral_slope',
 'spectral_tilt',
 'cepstral_peak_prominence',
 'mean_f1_loc',
 'std_f1_loc',
 'mean_b1_loc',
 'std_b1_loc',
 'mean_f2_loc',
 'std_f2_loc',
 'mean_b2_loc',
 'std_b2_loc',
 'spectral_gravity',
 'spectral_std_dev',
 'spectral_skewness',
 'spectral_kurtosis']

### Rounding stuff for the mean (SD) table in results

In [28]:
# round MDC data
mdc_female_ris_rounded = mdc_female_reference_intervals.copy()
mdc_male_ris_rounded = mdc_male_reference_intervals.copy()

mdc_female_ris_rounded['RI_lower_limit'] = mdc_female_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))
mdc_male_ris_rounded['RI_lower_limit'] = mdc_male_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))

mdc_female_ris_rounded['RI_upper_limit'] = mdc_female_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))
mdc_male_ris_rounded['RI_upper_limit'] = mdc_male_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))

# round CLAC data
clac_female_ris_rounded = clac_female_reference_intervals.copy()
clac_male_ris_rounded = clac_male_reference_intervals.copy()

clac_female_ris_rounded['RI_lower_limit'] = clac_female_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))
clac_male_ris_rounded['RI_lower_limit'] = clac_male_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))

clac_female_ris_rounded['RI_upper_limit'] = clac_female_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))
clac_male_ris_rounded['RI_upper_limit'] = clac_male_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))

# round common voice data
common_female_ris_rounded = common_female_reference_intervals.copy()
common_male_ris_rounded = common_male_reference_intervals.copy()

common_female_ris_rounded['RI_lower_limit'] = common_female_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))
common_male_ris_rounded['RI_lower_limit'] = common_male_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))

common_female_ris_rounded['RI_upper_limit'] = common_female_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))
common_male_ris_rounded['RI_upper_limit'] = common_male_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))

### Determine the deviation scores!

In [29]:
def score_ds_q123(value, q1, q3, median):
    """
    if feature in reference interval: score2 = 0
    otherwise, score2 will be the distance to the edge of th interval,
    normalized by the legth of the half interval. Notice that because we
    are not assuming a normal distribution, the upper and lower half-intervals
    may be different.
    """
    ri_length = np.abs(q3 - q1)/2
    if value > q3:
        return np.abs(q3 - value)/ri_length
   
    elif value < q1:
        return -np.abs(q1 - value)/ri_length
   
    else:
        return 0

In [30]:
import numpy as np

def calc_dev_scores(df, ri_df, feature_list):
    new_df = df.copy()
    
    for feature in feature_list:
        q3 = ri_df.loc[ri_df["feature"] == feature, "q3"].iloc[0]
        q1 = ri_df.loc[ri_df["feature"] == feature, "q1"].iloc[0]
        median = ri_df.loc[ri_df["feature"] == feature, "median"].iloc[0]

        new_df[feature] = new_df[feature].apply(lambda x: score_ds_q123(x, q1, q3, median))
    return new_df

In [31]:
mdc_male_reference_intervals['feature'] = mdc_male_reference_intervals['feature'].apply(lambda x: x.lower())
mdc_female_reference_intervals['feature'] = mdc_female_reference_intervals['feature'].apply(lambda x: x.lower())

common_male_reference_intervals['feature'] = common_male_reference_intervals['feature'].apply(lambda x: x.lower())
common_female_reference_intervals['feature'] = common_female_reference_intervals['feature'].apply(lambda x: x.lower())

clac_male_reference_intervals['feature'] = clac_male_reference_intervals['feature'].apply(lambda x: x.lower())
clac_female_reference_intervals['feature'] = clac_female_reference_intervals['feature'].apply(lambda x: x.lower())

In [32]:
mdc_males_radar_dev = calc_dev_scores(mdc_males_radar, mdc_male_reference_intervals, speech_features)
mdc_females_radar_dev = calc_dev_scores(mdc_females_radar, mdc_female_reference_intervals, speech_features)

common_males_radar_dev = calc_dev_scores(common_males_radar, common_male_reference_intervals, speech_features)
common_females_radar_dev = calc_dev_scores(common_females_radar, common_female_reference_intervals, speech_features)

clac_males_radar_dev = calc_dev_scores(clac_males_radar, clac_male_reference_intervals, speech_features)
clac_females_radar_dev = calc_dev_scores(clac_females_radar, clac_female_reference_intervals, speech_features)

<positron-console-cell-32>:14: RuntimeWarning: divide by zero encountered in scalar divide
<positron-console-cell-32>:11: RuntimeWarning: divide by zero encountered in scalar divide


Common voice pause features need to be removed

In [33]:
common_males_radar_dev.drop(["phonation_ratio", "pause_rate", "mean_pause_duration"], axis=1, inplace=True)
common_females_radar_dev.drop(["phonation_ratio", "pause_rate", "mean_pause_duration"], axis=1, inplace=True)

In [34]:
mdc_males_radar_dev

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,0.476147,0.000000,0.804976,-1.668295,0.000000,0.000000,0.300975,0.000000,-0.025543,-0.114357,-0.465135,0.000000,-0.473948,-0.938910,0.000000,0.000000,0.000000,-0.410032,0.000000,0.000000,0.000000,0.000000,0.000000,1.070685,0.780836
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,0.000000,0.000000,0.000000,-0.060290,0.000000,0.804993,0.000000,1.252063,-0.031127,0.000000,0.000000,-2.022302,0.000000,0.605324,0.000000,2.014562,0.000000,-0.706013,-1.736432,0.000000,0.026449,0.000000,0.000000,-0.787552,-0.574804
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,0.427502,-0.680372,2.380254,-3.101706,-2.086183,1.229235,0.589749,1.047718,-0.029120,0.000000,0.000000,0.316056,0.000000,0.000000,0.000000,2.159943,0.048745,0.000000,-1.004485,0.000000,0.000000,0.162291,0.182921,0.000000,0.000000
66,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-12,RADAR-MDD-KCL-s1,11,26,0,21,170,0.283907,-0.038339,1.549582,-1.221388,-1.276241,-0.172965,0.000000,-3.496928,-0.019256,-0.170270,0.000000,0.000000,-1.305150,0.000000,0.762059,2.345368,1.560159,0.510841,-0.852364,0.784912,0.000000,-0.733881,0.000000,0.000000,0.000000
67,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-26,RADAR-MDD-KCL-s1,6,26,0,21,170,0.111628,-0.306524,1.551933,-1.593724,-0.921891,-0.381707,-0.363535,0.000000,-0.028499,-0.461579,0.000000,0.000000,-1.397715,0.000836,1.378388,2.069460,2.018763,2.446776,-2.930345,1.373125,0.000000,-0.542518,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-08,RADAR-MDD-KCL-s1,14,51,0,12,165,0.000000,-0.785127,0.668822,-0.275910,-0.289124,-0.535684,0.000000,-1.360627,-0.022688,0.000000,-0.772840,-0.273624,-0.742419,0.000000,0.000000,1.877059,0.607479,0.000000,0.000000,1.079449,0.358135,-1.121107,0.000000,-0.244422,-0.324118
4217,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-22,RADAR-MDD-KCL-s1,13,51,0,12,165,-0.310116,-2.438638,0.000000,-0.001779,0.000000,-0.061409,0.000000,-0.831708,-0.025527,0.000000,0.000000,0.000000,0.000000,-0.606962,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.565795,0.000000,0.000000,0.000000
4218,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-05,RADAR-MDD-KCL-s1,11,51,0,12,165,0.305664,0.000000,1.461324,-2.839648,0.000000,-0.356608,0.000000,-4.590559,-0.020500,0.000000,0.000000,0.000000,-1.215635,0.000000,2.551845,2.374501,0.735777,0.965616,0.050614,1.666312,0.233607,-1.160706,0.000000,0.131035,0.000000
4219,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-19,RADAR-MDD-KCL-s1,16,51,0,12,165,0.000000,0.000000,0.000000,0.000000,0.000000,-0.168528,-0.035661,-3.618942,-0.019553,0.776954,-0.085448,0.000000,-0.852486,0.000000,1.560916,2.382537,1.094551,0.000000,1.953001,4.809063,1.662701,-2.065695,-0.005815,0.736954,0.000000


### combining all the data

In [35]:
mdc_radar_features = pd.concat([mdc_females_radar_dev, mdc_males_radar_dev])
clac_radar_features = pd.concat([clac_females_radar_dev, clac_males_radar_dev])
common_radar_features = pd.concat([common_females_radar_dev, common_males_radar_dev])

In [36]:
location_to_save_data = "../data/prediction_data/"

In [37]:
mdc_radar_features.to_csv(location_to_save_data+"mdc_radar_features.csv")
clac_radar_features.to_csv(location_to_save_data+"clac_radar_features.csv")
common_radar_features.to_csv(location_to_save_data+"common_radar_features.csv")

### How many people deviate for each feature?

In [69]:
features_of_interest = ["speaking_rate", "mean_f1_loc", "mean_f2_loc", "mean_f0", "spectral_gravity"]

In [70]:
current_df = mdc_radar_features

for col in features_of_interest:
    count_inrange = (current_df[col] == 0).sum()
    count_negative = (current_df[col] < 0).sum()
    count_positive = (current_df[col] > 0).sum()
    print(f"{col}\nin range: {count_inrange}\nnegative: {count_negative}\npositive: {count_positive}\n\n") 

speaking_rate
in range: 1933
negative: 312
positive: 1995


mean_f1_loc
in range: 2259
negative: 890
positive: 1091


mean_f2_loc
in range: 1901
negative: 1116
positive: 1223


mean_f0
in range: 2110
negative: 1249
positive: 881


spectral_gravity
in range: 2141
negative: 1627
positive: 472


speaking_rate
in range: 1933
negative: 312
positive: 1995


mean_f1_loc
in range: 2259
negative: 890
positive: 1091


mean_f2_loc
in range: 1901
negative: 1116
positive: 1223


mean_f0
in range: 2110
negative: 1249
positive: 881


spectral_gravity
in range: 2141
negative: 1627
positive: 472




### Age descriptives

In [71]:
mdc_radar_features["age"].describe()

count    4240.000000
mean       48.371698
std        15.362923
min        18.000000
25%        34.000000
50%        53.000000
75%        62.000000
max        80.000000
Name: age, dtype: float64

In [72]:
clac_radar_features["age"].describe()

count    4240.000000
mean       48.371698
std        15.362923
min        18.000000
25%        34.000000
50%        53.000000
75%        62.000000
max        80.000000
Name: age, dtype: float64

### number of RADAR observations etc.

In [75]:
# Count the number of observations per group
group_counts = clac_radar_features.groupby('pid').size()

# Calculate mean and standard deviation of the number of observations per group
mean_count = group_counts.mean()
std_dev_count = group_counts.std()

print(mean_count)
print(std_dev_count)
print(len(clac_radar_features))

15.588235294117647
10.430207753189322
4240
15.588235294117647
10.430207753189322
4240


### Gender distribution in RADAR

In [48]:
mdc_radar_features.head()

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-08-28,RADAR-MDD-KCL-s1,15,61,1,17,170,0.000000,0.000000,0.078133,0.000000,-0.156474,-1.079457,0.000000,-0.645626,-1.119524,0.00000,0.000000,0.075820,-0.677967,0.000000,2.034834,4.637065,0.000000,0.000000,-1.880249,0.000000,-0.828264,-1.755822,0.0,2.765793,1.188751
2,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-10-23,RADAR-MDD-KCL-s1,12,61,1,17,170,0.004396,0.000000,0.806427,-0.563334,-0.345473,-1.965223,-0.212725,0.000000,-1.120296,-0.12116,-1.455995,0.670738,-1.563093,0.000000,1.373056,3.826468,0.000000,0.000000,-0.417183,1.604224,-1.558878,-0.771859,0.0,1.671019,0.660698
3,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-12-04,RADAR-MDD-KCL-s1,6,61,1,17,170,0.000000,-0.051830,1.084159,-0.550633,-0.912470,-1.162549,0.000000,0.000000,-1.124543,0.00000,-1.127605,-0.632039,-0.657503,2.072945,2.098234,4.235902,1.038265,-0.170176,-0.867325,0.000000,-1.246056,-1.205538,0.0,0.618296,0.482388
4,00d1d60a-15cf-481b-8264-91705b6f8d97,2019-12-18,RADAR-MDD-KCL-s1,9,61,1,17,170,0.000000,-0.064468,0.000000,0.000000,-0.314546,-1.557545,0.000000,0.000000,-1.126004,0.00000,-0.859537,-0.687045,-1.052914,1.986542,2.537278,5.243086,2.071602,0.000000,-0.652433,1.205413,0.000000,-1.962990,0.0,1.525347,1.331167
5,00d1d60a-15cf-481b-8264-91705b6f8d97,2020-01-01,RADAR-MDD-KCL-s1,11,61,1,17,170,0.000000,0.000000,0.425921,0.000000,-0.637562,-1.086912,0.095455,0.000000,-1.127035,0.00000,-1.390508,0.000000,-0.808436,0.000000,1.159323,2.945559,1.089634,0.000000,-0.775992,0.447378,-0.573145,-1.269801,0.0,3.159095,3.169213


In [59]:
mdc_radar_features.groupby("pid").sample(n=1, random_state=30)["gender"].value_counts()

gender
1    213
0     59
Name: count, dtype: int64

## exploring issues with common voice

In [38]:
import math

In [39]:
mdc_females_radar_dev.isin([np.inf, -np.inf]).sum()

pid                         0
pdate                       0
study_site                  0
phq8                        0
age                         0
gender                      0
education_years             0
height                      0
speaking_rate               0
articulation_rate           0
phonation_ratio             0
pause_rate                  0
mean_pause_duration         0
mean_f0                     0
stdev_f0_semitone           0
mean_db                     0
range_ratio_db              0
hnr_db                      0
spectral_slope              0
spectral_tilt               0
cepstral_peak_prominence    0
mean_f1_loc                 0
std_f1_loc                  0
mean_b1_loc                 0
std_b1_loc                  0
mean_f2_loc                 0
std_f2_loc                  0
mean_b2_loc                 0
std_b2_loc                  0
spectral_gravity            0
spectral_std_dev            0
spectral_skewness           0
spectral_kurtosis           0
dtype: int

In [40]:
common_males_radar_dev.isin([np.inf, -np.inf]).sum()

pid                         0
pdate                       0
study_site                  0
phq8                        0
age                         0
gender                      0
education_years             0
height                      0
speaking_rate               0
articulation_rate           0
mean_f0                     0
stdev_f0_semitone           0
mean_db                     0
range_ratio_db              0
hnr_db                      0
spectral_slope              0
spectral_tilt               0
cepstral_peak_prominence    0
mean_f1_loc                 0
std_f1_loc                  0
mean_b1_loc                 0
std_b1_loc                  0
mean_f2_loc                 0
std_f2_loc                  0
mean_b2_loc                 0
std_b2_loc                  0
spectral_gravity            0
spectral_std_dev            0
spectral_skewness           0
spectral_kurtosis           0
dtype: int64

In [41]:
common_males_radar_dev

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,0.000000,0.000000,0.000000,0.000000,0.000000,-0.373827,0.000000,-0.568201,0.000000,-0.342997,-0.676315,0.000000,0.000000,0.000000,-0.766387,0.000000,0.000000,0.000000,0.000000,-0.013394,0.726830,0.972270
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,0.000000,0.000000,0.000000,0.000000,0.000000,-0.382636,0.000000,-0.101256,-1.711579,0.000000,0.000000,0.000000,0.598764,0.000000,-0.939727,-0.492176,0.000000,0.000000,0.000000,0.000000,-0.317617,-0.099161
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,0.000000,0.000000,0.000000,0.007449,0.000000,-0.379470,0.000000,0.000000,0.146857,0.000000,0.000000,0.000000,0.705139,0.000000,-0.432845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
66,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-12,RADAR-MDD-KCL-s1,11,26,0,21,170,0.000000,0.000000,-0.722018,0.000000,0.000000,-0.363909,0.000000,0.000000,0.000000,-1.151521,0.000000,0.011776,0.840812,0.511562,0.000000,0.000000,0.325474,0.000000,-0.422874,0.000000,0.000000,0.000000
67,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-26,RADAR-MDD-KCL-s1,6,26,0,21,170,0.000000,0.000000,-0.888754,-0.268523,0.000000,-0.378490,0.000000,0.000000,0.000000,-1.241561,0.000000,0.423325,0.638933,0.784195,0.077978,-1.350594,0.816066,0.000000,-0.290178,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-08,RADAR-MDD-KCL-s1,14,51,0,12,165,-0.135740,0.000000,-1.011746,0.000000,0.000000,-0.369325,0.000000,-0.845381,-0.156344,-0.604143,0.000000,0.000000,0.498154,0.000000,-0.280651,0.000000,0.571128,0.021118,-0.691387,-0.141410,0.000000,0.000000
4217,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-22,RADAR-MDD-KCL-s1,13,51,0,12,165,-1.182989,-0.375530,-0.632912,0.000000,0.000000,-0.373803,0.067019,0.000000,0.000000,0.000000,-0.467133,0.000000,0.000000,0.000000,-0.443990,0.000000,0.000000,0.000000,-0.306319,0.000000,0.000000,0.000000
4218,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-05,RADAR-MDD-KCL-s1,11,51,0,12,165,0.000000,0.000000,-0.868706,0.000000,-0.233761,-0.365873,0.000000,0.000000,0.000000,-1.064448,0.000000,1.206892,0.862129,0.021478,0.000000,0.230690,1.060594,0.000000,-0.718846,0.000000,0.000000,0.000000
4219,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-19,RADAR-MDD-KCL-s1,16,51,0,12,165,-0.358391,0.000000,-0.718474,-0.015844,0.000000,-0.364378,0.835253,-0.226181,0.000000,-0.711208,0.000000,0.545207,0.868009,0.234764,0.000000,1.598496,3.681766,0.770579,-1.346390,-0.353406,0.463491,0.000000


### Exploring issues with common voice

phonation_ratio             941
pause_rate                  941
mean_pause_duration         941

In [42]:
common_males_radar

,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,-0.164287,0.313886,-0.987676,0.450190,2.050753,-0.207209,0.315335,0.228333,0.006066,-0.395138,-1.059681,-0.270764,-0.957684,-0.995585,-0.410773,-0.471681,-0.420512,-1.157530,-0.037092,0.130628,0.082745,-0.408745,-0.716608,0.953301,0.720419
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,-0.536536,0.227596,-1.672166,0.992331,1.904135,0.156717,0.130943,0.620540,0.006032,-0.017441,-0.748257,-1.856079,-0.120344,0.217749,-0.284841,0.835949,0.266018,-1.270539,-1.010216,0.097432,0.512133,0.032348,-0.392289,-0.834398,-0.637277
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,-0.185778,-0.290310,0.194138,-0.033086,0.545993,0.312775,0.436235,0.585477,0.006044,0.270712,0.041817,0.745407,-0.446507,0.001231,-0.577634,0.898923,0.340751,-0.940075,-0.660540,-0.241978,-0.143604,0.172738,0.077374,0.252196,-0.305830
66,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-12,RADAR-MDD-KCL-s1,11,26,0,21,170,-0.249216,-0.076442,-0.429053,0.600865,0.916395,-0.938728,-0.620109,-0.537482,0.006103,-0.425039,-0.153093,-0.253387,-1.511384,-0.266796,0.528460,0.979242,0.939991,-0.042299,-0.587867,0.795445,0.218334,-0.829846,-0.692163,0.065965,-0.195335
67,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-26,RADAR-MDD-KCL-s1,6,26,0,21,170,-0.325327,-0.165778,-0.427290,0.475332,1.078446,-1.015514,-0.800210,0.179299,0.006048,-0.580822,-0.475728,0.416292,-1.573046,0.010808,0.787749,0.859729,1.121817,0.696868,-1.580587,1.116884,0.389492,-0.763601,-0.504736,0.442942,-0.185520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-08,RADAR-MDD-KCL-s1,14,51,0,12,165,-0.723459,-0.325205,-1.089822,0.919634,1.367823,-1.072155,-0.346024,-0.170926,0.006083,0.031340,-1.244542,-0.807488,-1.136524,-0.482332,-0.454089,0.776388,0.562276,-0.840852,0.481674,0.956399,0.639673,-0.963894,-0.793781,-0.582741,-0.535849
4217,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-22,RADAR-MDD-KCL-s1,13,51,0,12,165,-1.395225,-0.876005,-1.751609,1.012058,1.958152,-0.897693,-0.003431,-0.080172,0.006066,0.659850,-0.434584,-0.063487,-0.266951,-0.881946,-0.559293,-0.342813,-0.212027,-0.947342,0.560464,0.230935,0.321552,-0.771659,-0.523482,0.049967,-0.334926
4218,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-05,RADAR-MDD-KCL-s1,11,51,0,12,165,-0.239605,-0.033002,-0.495267,0.055267,2.351705,-1.006282,0.087213,-0.725132,0.006096,-0.062224,-0.536876,0.519576,-1.451754,-0.021354,1.281420,0.991861,0.613143,0.131340,0.798980,1.277101,0.591789,-0.977603,-0.566367,0.517918,0.122575
4219,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-19,RADAR-MDD-KCL-s1,16,51,0,12,165,-0.866280,-0.056971,-1.948971,1.093352,2.036863,-0.937096,-0.662939,-0.558418,0.006101,1.151049,-0.831575,0.330491,-1.209845,-0.612864,0.864538,0.995342,0.755389,-0.287135,1.707813,2.994509,1.141306,-1.290888,-0.921580,0.798668,0.200358


In [43]:

common_males_radar_dev = calc_dev_scores(common_males_radar, common_male_reference_intervals, speech_features)

<positron-console-cell-43>:14: RuntimeWarning: divide by zero encountered in scalar divide
<positron-console-cell-43>:11: RuntimeWarning: divide by zero encountered in scalar divide


In [44]:
common_males_radar_dev


,pid,pdate,study_site,phq8,age,gender,education_years,height,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
54,02919d70-343c-4efa-97c0-c34bf85546f0,2020-02-07,RADAR-MDD-KCL-s1,9,58,0,20,168,0.000000,0.000000,-inf,inf,inf,0.000000,0.000000,0.000000,-0.373827,0.000000,-0.568201,0.000000,-0.342997,-0.676315,0.000000,0.000000,0.000000,-0.766387,0.000000,0.000000,0.000000,0.000000,-0.013394,0.726830,0.972270
55,02919d70-343c-4efa-97c0-c34bf85546f0,2020-04-30,RADAR-MDD-KCL-s1,5,58,0,20,168,0.000000,0.000000,-inf,inf,inf,0.000000,0.000000,0.000000,-0.382636,0.000000,-0.101256,-1.711579,0.000000,0.000000,0.000000,0.598764,0.000000,-0.939727,-0.492176,0.000000,0.000000,0.000000,0.000000,-0.317617,-0.099161
56,02919d70-343c-4efa-97c0-c34bf85546f0,2020-07-23,RADAR-MDD-KCL-s1,4,58,0,20,168,0.000000,0.000000,-inf,inf,inf,0.000000,0.007449,0.000000,-0.379470,0.000000,0.000000,0.146857,0.000000,0.000000,0.000000,0.705139,0.000000,-0.432845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
66,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-12,RADAR-MDD-KCL-s1,11,26,0,21,170,0.000000,0.000000,-inf,inf,inf,-0.722018,0.000000,0.000000,-0.363909,0.000000,0.000000,0.000000,-1.151521,0.000000,0.011776,0.840812,0.511562,0.000000,0.000000,0.325474,0.000000,-0.422874,0.000000,0.000000,0.000000
67,03d596f1-cf28-4204-a4c0-f2c08a3fe00b,2019-12-26,RADAR-MDD-KCL-s1,6,26,0,21,170,0.000000,0.000000,-inf,inf,inf,-0.888754,-0.268523,0.000000,-0.378490,0.000000,0.000000,0.000000,-1.241561,0.000000,0.423325,0.638933,0.784195,0.077978,-1.350594,0.816066,0.000000,-0.290178,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-08,RADAR-MDD-KCL-s1,14,51,0,12,165,-0.135740,0.000000,-inf,inf,inf,-1.011746,0.000000,0.000000,-0.369325,0.000000,-0.845381,-0.156344,-0.604143,0.000000,0.000000,0.498154,0.000000,-0.280651,0.000000,0.571128,0.021118,-0.691387,-0.141410,0.000000,0.000000
4217,feef4454-7797-45fc-8d86-597c0fe204a1,2020-12-22,RADAR-MDD-KCL-s1,13,51,0,12,165,-1.182989,-0.375530,-inf,inf,inf,-0.632912,0.000000,0.000000,-0.373803,0.067019,0.000000,0.000000,0.000000,-0.467133,0.000000,0.000000,0.000000,-0.443990,0.000000,0.000000,0.000000,-0.306319,0.000000,0.000000,0.000000
4218,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-05,RADAR-MDD-KCL-s1,11,51,0,12,165,0.000000,0.000000,-inf,inf,inf,-0.868706,0.000000,-0.233761,-0.365873,0.000000,0.000000,0.000000,-1.064448,0.000000,1.206892,0.862129,0.021478,0.000000,0.230690,1.060594,0.000000,-0.718846,0.000000,0.000000,0.000000
4219,feef4454-7797-45fc-8d86-597c0fe204a1,2021-01-19,RADAR-MDD-KCL-s1,16,51,0,12,165,-0.358391,0.000000,-inf,inf,inf,-0.718474,-0.015844,0.000000,-0.364378,0.835253,-0.226181,0.000000,-0.711208,0.000000,0.545207,0.868009,0.234764,0.000000,1.598496,3.681766,0.770579,-1.346390,-0.353406,0.463491,0.000000


In [80]:
common_male_reference_intervals

,feature,RI_lower_limit,RI_upper_limit,CI_for_lower_limit,CI_for_upper_limit,ci_lower_width/ri_width,ci_upper_width/ri_width,valid_CI_lower_limit,valid_CI_upper_limit,median,q1,q3
0,speaking_rate,-1.951074,2.027438,"(-2.0430659112053693, -1.889482685372107)","(1.873513559336328, 2.092349649861843)",0.038603,0.055004,True,True,0.008782,-0.636388,0.646525
1,articulation_rate,-1.981310,2.037868,"(-2.0592242557059404, -1.9100428425836777)","(1.9540618612540428, 2.1225700493751223)",0.037117,0.041926,True,True,-0.008460,-0.635566,0.644968
2,phonation_ratio,-2.871552,0.470464,"(-3.035941580347264, -2.7171930131683935)","(0.4704640709319927, 0.4704640709319927)",0.095376,0.000000,True,True,0.470464,0.470464,0.470464
3,pause_rate,-0.514434,2.774186,"(-0.5144338738173846, -0.5144338738173846)","(2.6455941455382828, 2.884351818111822)",0.000000,0.072601,True,True,-0.514434,-0.514434,-0.514434
4,mean_pause_duration,-0.371731,1.758616,"(-0.3717307864738716, -0.3717307864738716)","(1.6679626449525968, 1.8945952484444262)",0.000000,0.106383,True,True,-0.371731,-0.371731,-0.371731
5,mean_f0,-1.218033,2.691959,"(-1.2405759347778662, -1.1975793830403305)","(2.538344863873357, 2.8603228740704423)",0.010997,0.082347,True,True,-0.204895,-0.606222,0.314826
6,stdev_f0_semitone,-1.374271,2.340322,"(-1.4010759606283691, -1.3483300121673998)","(2.168706216114545, 2.6784722304568445)",0.014200,0.137233,True,True,-0.154958,-0.654332,0.432188
7,mean_db,-2.313984,1.546579,"(-2.439470698329487, -2.179544127804986)","(1.4929973472486828, 1.5972932205455093)",0.067329,0.027016,True,True,0.144840,-0.571965,0.738492
8,range_ratio_db,-0.092162,0.086869,"(-0.11604931780547383, -0.08002609041446423)","(0.07646005985987181, 0.10768596372397772)",0.201212,0.174416,False,True,0.009798,0.007471,0.014990
9,hnr_db,-1.795911,2.151837,"(-1.8435504548248862, -1.7533565587387456)","(2.0566901435949916, 2.2250389308104377)",0.022847,0.042644,True,True,-0.070469,-0.661778,0.616998


In [45]:
def calc_dev_scores(df, ri_df, feature_list):
    new_df = df.copy()
    
    for feature in feature_list:
        q3 = ri_df.loc[ri_df["feature"] == feature, "q3"].iloc[0]
        print(q3)
        q1 = ri_df.loc[ri_df["feature"] == feature, "q1"].iloc[0]
        print(q1)
        median = ri_df.loc[ri_df["feature"] == feature, "median"].iloc[0]
        print(median)

        new_df[feature] = new_df[feature].apply(lambda x: score_ds_q123(x, q1, q3, median))
    return new_df

In [46]:
def score_ds_q123(value, q1, q3, median):
    """
    if feature in reference interval: score2 = 0
    otherwise, score2 will be the distance to the edge of th interval,
    normalized by the legth of the half interval. Notice that because we
    are not assuming a normal distribution, the upper and lower half-intervals
    may be different.
    """
    ri_length = np.abs(q3 - q1)/2
    if value > q3:
        return np.abs(q3 - value)/ri_length
   
    elif value < q1:
        return -np.abs(q1 - value)/ri_length
   
    else:
        return 0

In [47]:
common_males_radar_dev = calc_dev_scores(common_males_radar, common_male_reference_intervals, ["phonation_ratio"])

0.4704640709319927
0.4704640709319927
0.4704640709319927
0.4704640709319927
0.4704640709319927
0.4704640709319927


<positron-console-cell-47>:14: RuntimeWarning: divide by zero encountered in scalar divide
<positron-console-cell-47>:11: RuntimeWarning: divide by zero encountered in scalar divide
